**jumbo rolls are folded in**

**changeover between jumbos are folded in**

**Minimize knife changes folded in**

need to ..

* show when an order in the future has been fullfilled.

* look forward in schedule to optimize new/better layouts

* make layout table editable

* speed test algorithm

* LM get rid of decimals

check work orders to full jumbo

other 

* student help

For next time: the time (knife changes) scheduler isn't working for Quality/SM/cycle 1. May be due to order being shorter - there being some mismatch between jumbos and actual LM quatnityt. find_optimum is finding the right number of jumbos, display_layout is creating the correct layout. Sometimes a bug is a single keystroke. the issue was a using > instead of >=. we made the **exact** right amount of doffs

# Simple Genetic Algorithm to Optimize Deckle Loss

In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from genetic import *
from IPython.display import clear_output
import time
import datetime
import re

In [3]:
df = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')

# the input schedule appears as a data table on the dashboard
# this is used to select what to deckle
df_input_schedule = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')
df_input_schedule.insert(1, 'Technology', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'tech')))
df_input_schedule.insert(2, 'Color', df_input_schedule['Description'].apply(lambda x: x.split(';')[1] if type(x) == str else None))
df_input_schedule.insert(3, 'Width', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'width')))
df_input_schedule = df_input_schedule[[col for col in df_input_schedule.columns if 'Unnamed' not in str(col)]]
df_input_schedule['Total LM Order QTY'] = df_input_schedule['Total LM Order QTY'].round(1)

In [4]:
temp = df_input_schedule[['Customer Name', 'Technology', 'Color', 'Width', 'CYCLE / BUCKET',
                          'Description', 'Total LM Order QTY',
                          'Scheduled Ship Date', 'Date order is complete']]
temp.loc[:,'Scheduled Ship Date'] = pd.to_datetime(temp['Scheduled Ship Date'], errors='coerce')
temp.loc[:,"Scheduled Ship Date"] = temp["Scheduled Ship Date"].dt.strftime("%Y-%m-%d")

temp.loc[:,"Date order is complete"] = temp["Date order is complete"].dt.round('T')


/home/wab665/anaconda3/envs/py37/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
temp.head()

,Customer Name,Technology,Color,Width,CYCLE / BUCKET,Description,Total LM Order QTY,Scheduled Ship Date,Date order is complete
0,P & G,SAM,WHITE,208,CYCLE 1,SAM0130UN0208;WHITE;300,369822.5,2020-07-21,2020-07-20 13:05:00
1,P & G,SAM,WHITE,170,CYCLE 1,SAM0130UN0170;WHITE;300,2262443.4,2020-07-21,2020-07-20 16:01:00
2,P & G,SAM,WHITE,234,CYCLE 1,SAM0130UN0234;WHITE;300,2005259.7,2020-07-22,2020-07-20 19:35:00
3,P & G,SAM,WHITE,234,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,2020-07-22,2020-07-20 23:23:00
4,P & G,SAM,WHITE,234,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,2020-07-22,2020-07-21 03:11:00


In [6]:
# temp = temp[temp['Scheduled Ship Date'].apply(lambda x: type(x) == int)]                          
temp_json = temp.to_json()

In [7]:
temp2 = pd.read_json(temp_json, 
                     convert_dates=['Scheduled Ship Date', 'Date order is complete'])

## to_datetime and to_json use default units ms and ns, respectively
temp2['Scheduled Ship Date'] = pd.to_datetime(temp2['Scheduled Ship Date'], 
                                              errors='coerce', unit='ms')

In [8]:
temp2["Scheduled Ship Date"] = temp2["Scheduled Ship Date"].dt.strftime("%Y-%m-%d")

In [9]:
for i, x in enumerate(temp2['Scheduled Ship Date']):
    if type(x) == pd._libs.tslibs.timestamps.Timestamp:
        temp2.loc[i, 'Scheduled Ship Date'] = datetime.datetime.strftime(x, '%Y-%m-%d')

In [10]:
temp2

,Customer Name,Technology,Color,Width,CYCLE / BUCKET,Description,Total LM Order QTY,Scheduled Ship Date,Date order is complete
0,P & G,SAM,WHITE,208.0,CYCLE 1,SAM0130UN0208;WHITE;300,369822.5,2020-07-21,2020-07-20 13:05:00
1,P & G,SAM,WHITE,170.0,CYCLE 1,SAM0130UN0170;WHITE;300,2262443.4,2020-07-21,2020-07-20 16:01:00
2,P & G,SAM,WHITE,234.0,CYCLE 1,SAM0130UN0234;WHITE;300,2005259.7,2020-07-22,2020-07-20 19:35:00
3,P & G,SAM,WHITE,234.0,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,2020-07-22,2020-07-20 23:23:00
4,P & G,SAM,WHITE,234.0,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,2020-07-22,2020-07-21 03:11:00
...,...,...,...,...,...,...,...,...,...
355,CARDINAL HEALTH,SM,WHITE,575.0,CYCLE 6,SM10120UN0575;WHITE;300,18985507.2,NaT,2020-12-25 14:58:00
356,None,None,None,NaN,CYCLE 6,None,NaN,NaT,2020-12-25 15:28:00
357,P & G,SM,WHITE,80.0,CYCLE 6,SM30330UN0080;WHITE;600,28030303.0,NaT,2020-12-27 20:40:00
358,IRVING CONSUMER PRODUCTS,SM,WHITE,80.0,CYCLE 6,SM30330UN0080;WHITE;600,7575757.6,NaT,2020-12-28 11:03:00


In [11]:
### not currently used, grabs indicated information
customer_names = [i for i in df['Customer Name'].unique() if type(i) == str]
technologies = list(set([parse_description(i, 'tech') for i in df['Description'] 
                         if (type(i) == str)]))
technologies = [i for i in technologies if i]
colors = list(set([i[1] for i in list(df['Description']
                .str.split(';')) if type(i) == list]))
cycles = list(df['CYCLE / BUCKET'].unique())

In [12]:
customer = 'FIRST QUALITY PRODUCTS'
technology = 'SM' # Technology -> 
color = 'DENBLUE'
cycle = 'CYCLE 1'

# customer = 'P & G'
# technology = 'SAM'
# color = 'WHITE'
# cycle = 'CYCLE 1'

df_filtered = df.loc[df['Customer Name'] == customer]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(technology)]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(color)] #CYAN, TEAL
df_filtered = df_filtered.loc[df_filtered['CYCLE / BUCKET'] == cycle]
df_filtered = df_filtered.loc[df_filtered['Total LM Order QTY'] > 0]
df_filtered.insert(0, 'Block', 1)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['Order Number'] = df_filtered.index + 1

df_filtered['Width'] = pd.DataFrame(list(pd.DataFrame(list(df_filtered['Description']
                                                  .str.split(';')))[0].str.split('UN0')))[1]
df_filtered.head(10)

,Block,SPM375014 PROD. DATE,CYCLE / BUCKET,Customer Name,Item No,WO#,Description,Treatment,KG,Hours To Produce,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,% reclaim,Amount Repro needed,Type of Repro,Order Number,Width
0,1,2020-07-25 15:55:11.795,CYCLE 1,FIRST QUALITY PRODUCTS,1790263,82349245,SM30110UN0670;DENBLUE;600,UNTREATED,5500.0,3.806228,...,NaN,NaN,NaN,NaN,NaN,0.15,1981.588235,SMS,1,670
1,1,2020-07-25 19:43:34.217,CYCLE 1,FIRST QUALITY PRODUCTS,1790262,82349246,SM30110UN0510;DENBLUE;600,UNTREATED,5500.0,3.806228,...,NaN,NaN,NaN,NaN,NaN,0.15,1981.588235,SMS,2,510
2,1,2020-07-25 23:31:56.639,CYCLE 1,FIRST QUALITY PRODUCTS,1790261,82349250,SM30110UN0245;DENBLUE;600,UNTREATED,2200.0,1.522491,...,NaN,NaN,NaN,NaN,NaN,0.15,1981.588235,SMS,3,245
3,1,2020-07-26 01:03:17.608,CYCLE 1,FIRST QUALITY PRODUCTS,1789190,82327051,SM30110UN0440;DENBLUE;600,UNTREATED,6600.0,4.567474,...,NaN,NaN,NaN,NaN,NaN,0.15,1981.588235,SMS,4,440
4,1,2020-07-26 05:37:20.515,CYCLE 1,FIRST QUALITY PRODUCTS,1789191,82327039,SM30110UN0600;DENBLUE;600,UNTREATED,8800.0,6.089965,...,NaN,NaN,NaN,NaN,NaN,0.15,1981.588235,SMS,5,600
5,1,2020-07-26 11:42:44.390,CYCLE 1,FIRST QUALITY PRODUCTS,1755582,82321375,SM30110UN0454;DENBLUE;600,UNTREATED,6600.0,4.567474,...,NaN,NaN,NaN,NaN,NaN,0.15,1981.588235,SMS,6,454


In [14]:
### Detects if there is a break in the rolls coming of the
### line

prev_index = df_filtered.index[0] - 1
block = 1
for index in df_filtered.index:
    if index != prev_index + 1:
        block += 1
    df_filtered.loc[index, 'Block'] = block
    prev_index = index

In [15]:
list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))

[245, 440, 454, 510, 600, 670]

In [16]:
lm = list(df_filtered.groupby('Description')['Total LM Order QTY'].sum().values)
widths = list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
doffs_in_jumbo = 6
B = 4160
L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
neckins = []
for width in widths:
    if width < 170:
        neckin = 4
    elif width < 208:
        neckin = 5
    else:
        neckin = 7
    neckins.append(neckin)
w = list(np.array(widths) + np.array(neckin)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)

print('The important variables', end='\n\n')
print('widths: {} (mm)'.format(widths))
print('neck in: {} (mm)'.format(neckins))
print('L (m): {}'.format(L))
print('undeckled jumbos needed: {}'.format(q))

The important variables

widths: [245, 440, 454, 510, 600, 670] (mm)
neck in: [7, 7, 7, 7, 7, 7] (mm)
L (m): 102000
undeckled jumbos needed: [9, 14, 13, 10, 14, 8]


In [17]:
# check that s has proper amount of jumbos
from collections import Counter
Counter(s)

Counter({252: 9, 447: 14, 461: 13, 517: 10, 607: 14, 677: 8})

In [18]:
# customer = 'P & G'
# product = 'SAM' # Technology -> 
# color = 'WHITE'
# cycle = 'CYCLE 2'

# df = pd.read_excel('../data/berry/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
#                    sheet_name='Schedule')
# df = df.loc[df['Customer Name'] == customer]
# df = df.loc[df['Description'].str.contains(product)]
# df = df.loc[df['Description'].str.contains(color)] #CYAN, TEAL
# df = df.loc[df['CYCLE / BUCKET'] == cycle]
# df = df.reset_index(drop=True)
# df['Width'] = pd.DataFrame(list(pd.DataFrame(list(df['Description']
#                                                   .str.split(';')))[0].str.split('UN0')))[1]
# lm = list(df.groupby('Description')['Total LM Order QTY'].sum().values)
# widths = list(df.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
# doffs_in_jumbo = 6
# B = 4160
# L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
# w = list(np.array(widths) + np.array(neckin))
# q = [math.ceil(x/L) for x in lm]
# print("max doffs needed without stock cutting: {}".format(q))
# s = BinPackingExample(w, q)
# df_json = df.to_json()

if we gave each order its own entire row we would produce a lot of wasted material

# Run Deckle Optimizer

In [19]:
sol, loss = find_optimum(s, B, widths, neckins, max_unique_products=4, 
                         loss_target=12, max_doff_layouts=10)

gene time: 0.0, chromosome time: 1600944539.1, ffd time: 0.0
loss: 10.248, best loss: 10.248, steps: 1, gene count: 5, unique doffs: 9, max unique on doff: 3
                 Formula Doffs  Loss
0                  245x4     1  3152
1          245x4 + 440x7     1    23
2          440x7 + 454x2     1   109
3                  454x9     1    11
4          454x2 + 510x6     1   136
5          510x3 + 600x4     1   181
6          510x1 + 600x6     1     1
7  245x1 + 600x4 + 670x2     1   126
8                  670x6     1    98


In [21]:
# def find_optimum(s,
#              B,
#              widths,
#              neckin,
             
loss_target = 12
max_doff_layouts = 20
max_unique_products = 6
gene_count = 5
doff_min = 1


losses = []
solutions = []
best_loss = 100
step = 0
start_time = time.time()

while True:
    step += 1
    shuffle(s)
    remain = [B] #initialize list of remaining bin spaces
    sol = [[]]
    binlim = np.inf

    for item in s:#sorted(s, reverse=True): # iter through products
        for j,free in enumerate(remain): #
            if (free >= item) and (len(sol[j]) < binlim): # if theres room
                remain[j] -= item
                sol[j].append(item)
                break
        else: # at this step we need to double the previous (or mult. by min. doff count)
            sol.append([item]) #starts new list in sol list
            remain.append(B-item) #append a new bin and subtract the width of the new item

    genes = []

    df = pd.DataFrame(sol)
    df = df.fillna(0)
    dff = pd.DataFrame(df.groupby(list(df.columns)).size()).\
        rename(columns={0: 'freq'}).reset_index()
    dff = dff[dff.columns[:-1]]
    dff["Loss"] = B - dff.sum(axis=1)
    dff = dff.loc[dff.replace(0, np.nan, inplace=False).nunique(axis=1) <= max_unique_products]
    dff = dff.loc[dff['Loss'] < 100]
    dff = dff.sort_values('Loss').head(gene_count).reset_index(drop=True)

    dff = dff[dff.columns[:-1]]
    dff = dff.fillna(0)

    for row in dff.index:
        gene = list(dff.iloc[row].values)
        gene = [i for i in gene if i != 0]
        genes.append(gene)

    order_remaining = copy.copy(s)
    sol2 = []
    gene_time = time.time() - start_time

    while True:
        backup_order_remaining = copy.copy(order_remaining)
        try:
            new_gene = list(choice(genes))
        except:
            break
        if new_gene in sol2:
            sol2.append(new_gene)
            for item in new_gene:
                if item in order_remaining:
                    order_remaining.remove(item)
                elif item == 0:
                    pass
                else: # new gene over produces item
                    order_remaining = backup_order_remaining
                    sol2.remove(new_gene)
                    genes.remove(new_gene)
                    break
        else:
            check_pass = True
            for mult in range(doff_min):

                for item in new_gene:
                    if item in order_remaining:
                        order_remaining.remove(item)
                    elif item == 0:
                        pass
                    else: # new gene over produces item

                        check_pass = False
                        break
                else:

                    continue  # only executed if the inner loop did NOT break
                if check_pass == False:  # only executed if the inner loop DID break
                    order_remaining = backup_order_remaining
                    genes.remove(new_gene)

                    break
            if check_pass == True:
                for mult in range(doff_min):
                    sol2.append(new_gene)

    chrome_time = time.time() - gene_time
    #     if step > iterations:
    #         break
    remain2 = [B] #initialize list of remaining bin spaces
#         print(remain2)
    sol3 = [[]]
    binlim = np.inf
    doff_min = 2
    for item in sorted(order_remaining, reverse=True): # iter through products
        for j,free in enumerate(remain2): #
            if (free >= item) and (len(sol3[j]) < binlim): # if theres room,
                remain2[j] -= item # and we haven't reach bimlim
                sol3[j].append(item)
                break
        else: # at this step we need to double the previous (or mult. by min. doff count)
            sol3.append([item]) #starts new list in sol list
            remain2.append(B-item) #append a new bin and subtract the width of the new item
            # subtract
    # loss = sum(remain2) / np.sum(np.sum(sol3)) * 100
    ffd_time = time.time() - chrome_time
    sol_tot = sol2 + sol3
    space_avail = len(sol_tot) * B
    loss = (space_avail - np.sum(np.sum(sol_tot))) / space_avail * 100
    losses.append(loss)
    solutions.append(sol_tot)
    if loss < best_loss:
        best_solution = sol_tot
        best_loss = loss
        print(best_loss)

    if (loss < loss_target) and \
        (summarize_results(sol_tot, widths, neckins, B).shape[0] <
        (max_doff_layouts+1)) and \
        (all(pd.DataFrame(sol_tot).replace(0, np.nan, inplace=False)\
        .nunique(axis=1) <= max_unique_products)):
        break
# return sol_tot, loss

10.248397435897436


In [22]:
Counter(order_remaining)

Counter({252: 9, 677: 6, 607: 6, 461: 5, 447: 2})

In [23]:
def flatten(seq, container=None):
    if container is None:
        container = []

    for s in seq:
        try:
            iter(s)  # check if it's iterable
        except TypeError:
            container.append(s)
        else:
            flatten(s, container)

    return container

In [24]:
# check if find_optimum() creates the right number of jumbos

print(Counter(flatten(sol_tot))-Counter(s))
print((Counter(s)))
print(Counter(flatten(sol_tot)))

Counter()
Counter({447: 14, 607: 14, 461: 13, 517: 10, 252: 9, 677: 8})
Counter({607.0: 14, 447.0: 14, 461.0: 13, 517.0: 10, 252: 9, 677.0: 8})


# Build Layout for Dashboard View

In [25]:
sol

[[517, 461, 677, 447, 252, 607, 461, 677],
 [252, 447, 607, 607, 252, 252, 447, 252, 677, 252],
 [607, 461, 447, 517, 461, 607, 517, 447],
 [607, 517, 607, 677, 461, 447, 607],
 [607, 607, 517, 461, 447, 461, 447, 447],
 [461, 517, 447, 607, 677, 461, 252, 447, 252],
 [677, 461, 517, 461, 461, 677, 517, 252],
 [607, 517, 461, 447, 447, 517, 677, 447],
 [607, 607]]

In [26]:
layout_summary(sol, widths, neckins, B)

,0,1,2,3,4,5,6,7,8,9,Doffs,Loss
Doffs,,,,,,,,,,,,
1,245,440,600.0,600.0,245.0,245.0,440.0,245.0,670.0,245.0,1,115.0
1,454,510,440.0,600.0,670.0,454.0,245.0,440.0,245.0,NaN,1,39.0
1,510,454,670.0,440.0,245.0,600.0,454.0,670.0,NaN,NaN,1,61.0
1,600,454,440.0,510.0,454.0,600.0,510.0,440.0,NaN,NaN,1,96.0
1,600,510,454.0,440.0,440.0,510.0,670.0,440.0,NaN,NaN,1,40.0
1,600,510,600.0,670.0,454.0,440.0,600.0,NaN,NaN,NaN,1,237.0
1,600,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2946.0
1,600,600,510.0,454.0,440.0,454.0,440.0,440.0,NaN,NaN,1,166.0
1,670,454,510.0,454.0,454.0,670.0,510.0,245.0,NaN,NaN,1,137.0


# Run Master Schedule, Optimizing Either for On-Time Orders or Fastest Schedule

In [27]:
setup_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Deckle Parameters')
speed_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Product Parameters')

In [28]:
start_date_time = datetime.datetime(2020,8,1)
start_date_time

datetime.datetime(2020, 8, 1, 0, 0)

In [29]:
extras = pd.DataFrame(np.zeros(len(widths))).T
extras

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
master2 = make_layout_registrar(sol, widths, neckins)
master2

../utils.py:490: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




,245,440,454,510,600,670,freq
0,5.0,2.0,NaN,NaN,2.0,1.0,1
0,2.0,2.0,2.0,1.0,1.0,1.0,1
0,1.0,1.0,2.0,1.0,1.0,2.0,1
0,NaN,2.0,2.0,2.0,2.0,NaN,1
0,NaN,3.0,1.0,2.0,1.0,1.0,1
0,NaN,1.0,1.0,1.0,3.0,1.0,1
0,NaN,NaN,NaN,NaN,2.0,NaN,1
0,NaN,3.0,2.0,1.0,2.0,NaN,1
0,1.0,NaN,3.0,2.0,NaN,2.0,1


In [36]:
neckin=neckins
extras = pd.DataFrame(np.zeros(len(widths))).T
master2 = make_layout_registrar(sol, widths, neckin)
master2.columns = master2.columns.str.strip()
DEBUG=True
if DEBUG:
    print(master2.head(10))
extras.columns = master2.columns[:-1]
schedule = []
schedule_with_order_info = []
layout_pattern = 0
old_width = width = None
completed_orders = []
df_filtered['Order Number'] = df_filtered.index + 1


   245  440  454  510  600  670  freq
0  5.0  2.0  NaN  NaN  2.0  1.0     1
0  2.0  2.0  2.0  1.0  1.0  1.0     1
0  1.0  1.0  2.0  1.0  1.0  2.0     1
0  NaN  2.0  2.0  2.0  2.0  NaN     1
0  NaN  3.0  1.0  2.0  1.0  1.0     1
0  NaN  1.0  1.0  1.0  3.0  1.0     1
0  NaN  NaN  NaN  NaN  2.0  NaN     1
0  NaN  3.0  2.0  1.0  2.0  NaN     1
0  1.0  NaN  3.0  2.0  NaN  2.0     1


In [ ]:
sol

In [39]:

# def optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, speed_df,
#                   doffs_in_jumbo, start_date_time, 
neckin=neckins
objective='Time (Knife Changes)'
# objective='Late Orders'
DEBUG=True

#####################################################
# Step 1: Create Schedule According to Order Sequence
#####################################################
extras = pd.DataFrame(np.zeros(len(widths))).T
master2 = make_layout_registrar(sol, widths, neckin)
master2.columns = master2.columns.str.strip()
if DEBUG:
    print(master2.head(10))
extras.columns = master2.columns[:-1]
schedule = []
schedule_with_order_info = []
layout_pattern = 0
old_width = width = None
completed_orders = []
df_filtered['Order Number'] = df_filtered.index + 1

if objective == 'Late Orders':
    #### logic for jumbo rolls should go in first loop
    for row1 in df_filtered.index:
        clear_output(wait=True)
        ### select schedule data
        current_scheduled = df_filtered.iloc[row1][['Total LM Order QTY', 'Width', 'Scheduled Ship Date']]
        current_scheduled['Order Number'] = row1+1
        ship_date = current_scheduled['Scheduled Ship Date']
        doffs = math.ceil(current_scheduled['Total LM Order QTY'] / L) # QTY
        width = str(current_scheduled['Width'])
        if DEBUG:
            print(width)

        ### check if we've entered a new product
        ### no setup changes here, we need to see if the layout has changed
        if width == old_width:
            layout_pattern -= 1

        ### sort by the current scheduled width in master 2
        master2 = master2.sort_values(str(width), ascending=False)
        master2 = master2.reset_index(drop=True)
        if DEBUG:
            print(master2.head())

        ### calc how many new doffs need to be made from inventory
        target_doffs = extras.iloc[0][width]
        if DEBUG:
            print(pd.DataFrame(data=[[doffs, width, target_doffs]],
                               columns=['required doffs', 'width', 'doffs made']))
            print(doffs, width, target_doffs)

        ### only proceed if doffs are available in layout registrar
        ### and not in inventory
        if ((any(master2.loc[master2[width] > 0]['freq'] > 0))
            and (target_doffs <= doffs)):

        ### go through the rows in the master2 registrar and check for
        ### layout patterns that contain our current width

        ### Every row is a new layout unless the outer
        ### loop has incremented, then only maybe is it
        ### a new layout
            for row in master2.index:

                # if none of the width are in the layout then break out
                if np.isnan(master2.iloc[row][width]):
                    break
                layout_pattern += 1

                # looping through available doffs in the layout
                for count in range(master2.iloc[row]['freq'].astype(int)):

                    # add to target doffs
                    target_doffs += master2.iloc[row][width]

                    # add to number of times we've made this layout
                    master2.at[row, 'layout number'] = layout_pattern


                    new_layout_and_scheduled_product = pd.concat([master2.iloc[row],
                                                                  current_scheduled])
                    schedule_with_order_info.append(new_layout_and_scheduled_product)
                    schedule.append(master2.iloc[row])
                    master2.at[row, 'freq'] = master2.at[row, 'freq'] - 1

                    ### Tabluate the other widths that were made on the layout
                    for master_index in master2.iloc[row].index[:-2]:
                        if (math.isnan(master2.iloc[row][master_index]) == False):
                            ### tabulate extras for other widths in the layout
                            extras.iloc[0][master_index] = extras.iloc[0][master_index] +\
                                master2.iloc[row][master_index]
                            completed_orders.append
                    if target_doffs >= doffs:

                        ### add to registrar of completed orders.
                        completed_orders.append(current_scheduled['Order Number'])

                        ### now that we've finished this order, we want to see if we've completed any other orders as well


                        break
                ### for exiting nested for loop
                else:
                    continue
                break
        old_width = width
        extra = target_doffs - doffs
        extras.iloc[0][width] = extra
        if DEBUG:
            print(extras)

    sorted_schedule_with_order_info = pd.DataFrame(schedule_with_order_info)
    sorted_schedule_with_order_info = sorted_schedule_with_order_info.reset_index(drop=True)
    sorted_schedule = pd.DataFrame(schedule)
    sorted_schedule = sorted_schedule.reset_index(drop=True)
elif objective == "Time (Knife Changes)":
    # when we sort master, this will be similar to how we sort
    # according to the desirable width in the optimize for late
    # order algorithm. We will however do this for every width

    for width in widths:
        master2 = master2.sort_values(str(width), ascending=False)
        master2 = master2.reset_index(drop=True)

    # now we need to go through the schedule and match layouts
    # with orders

    # go through the layouts
    for row in master2.index:

        # go through doffs in the layout
        for count in range(master2.iloc[row]['freq'].astype(int)):

            # here we will grow our extras dataframe. We will use this
            # to check against orders that are fullfilled.

            ### Tabluate the other widths that were made on the layout
            for master_index in master2.iloc[row].index[:-1]: # no layout number col, -2 in other algorithm

                if (math.isnan(master2.iloc[row][master_index]) == False):

                    ### tabulate extras for other widths in the layout
                    extras.iloc[0][master_index] = extras.iloc[0][master_index] +\
                        master2.iloc[row][master_index]

            # go through the orders
            added_this_roll = []
            if df_filtered.shape[0] > 0:
                for row1 in df_filtered.index:
                    current_scheduled = df_filtered.iloc[row1][['Total LM Order QTY', 'Width',
                                                                     'Scheduled Ship Date',
                                                                     'Order Number']]
                    
                    order_number = current_scheduled['Order Number']
                    ship_date = current_scheduled['Scheduled Ship Date']
                    doffs = math.ceil(current_scheduled['Total LM Order QTY'] / L) # QTY
                    width = str(current_scheduled['Width'])
                    if extras[width][0] >= doffs:
                        

                        completed_orders.append(order_number)
                        extra = extras[width][0] - doffs
                        extras.iloc[0][width] = extra
                        new_layout_and_scheduled_product = pd.concat([master2.iloc[row],
                                                                      current_scheduled])
                        schedule_with_order_info.append(new_layout_and_scheduled_product)
                        added_this_roll.append(order_number)
            # if no order completed just append the layout
            if len(added_this_roll) == 0:
                schedule_with_order_info.append(master2.iloc[row])
            # if two orders completed append both
            elif len(added_this_roll) == 2:
                schedule_with_order_info[-1][-1] =  "{} and {}".format(added_this_roll[0], added_this_roll[1])
                schedule_with_order_info.pop(-2)
            elif len(added_this_roll) > 2:
                print('this is not tested')
                schedule_with_order_info[-1][-1] = str(added_this_roll).split('[')[-1].split(']')[0]
                # schedule_with_order_info.pop(-len(added_this_roll))
            df_filtered = df_filtered[~df_filtered['Order Number'].isin(completed_orders)]
            df_filtered = df_filtered.reset_index(drop=True)
            master2.loc[0, 'freq'] -= 1


    sorted_schedule_with_order_info = pd.DataFrame(schedule_with_order_info)
    sorted_schedule_with_order_info = sorted_schedule_with_order_info.reset_index(drop=True)
    sorted_schedule_with_order_info[['Total LM Order QTY',
   'Width', 'Scheduled Ship Date', 'Order Number']] = \
    sorted_schedule_with_order_info[['Total LM Order QTY',
   'Width', 'Scheduled Ship Date', 'Order Number']].fillna(method='bfill')


   245  440  454  510  600  670  freq
0  5.0  2.0  NaN  NaN  2.0  1.0     1
0  2.0  2.0  2.0  1.0  1.0  1.0     1
0  1.0  1.0  2.0  1.0  1.0  2.0     1
0  NaN  2.0  2.0  2.0  2.0  NaN     1
0  NaN  3.0  1.0  2.0  1.0  1.0     1
0  NaN  1.0  1.0  1.0  3.0  1.0     1
0  NaN  NaN  NaN  NaN  2.0  NaN     1
0  NaN  3.0  2.0  1.0  2.0  NaN     1
0  1.0  NaN  3.0  2.0  NaN  2.0     1
this is not tested


In [41]:
sorted_schedule_with_order_info

,245,440,454,510,600,670,freq,Total LM Order QTY,Width,Scheduled Ship Date,Order Number
0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,8.163265e+05,245,2020-08-15,1 and 3
1,1.0,NaN,3.0,2.0,NaN,2.0,1.0,8.163265e+05,245,2020-08-15,1 and 3
2,NaN,1.0,1.0,1.0,3.0,1.0,1.0,8.163265e+05,245,2020-08-15,1 and 3
3,5.0,2.0,NaN,NaN,2.0,1.0,1.0,8.163265e+05,245,2020-08-15,1 and 3
4,NaN,3.0,1.0,2.0,1.0,1.0,1.0,8.163265e+05,245,2020-08-15,1 and 3
5,2.0,2.0,2.0,1.0,1.0,1.0,1.0,8.163265e+05,245,2020-08-15,1 and 3
6,NaN,2.0,2.0,2.0,2.0,NaN,1.0,9.803922e+05,510,2020-08-15,2
7,NaN,3.0,2.0,1.0,2.0,NaN,1.0,9.803922e+05,510,2020-08-15,2
8,NaN,3.0,2.0,1.0,2.0,NaN,1.0,1.363636e+06,440,2020-08-23,4
9,NaN,3.0,2.0,1.0,2.0,NaN,1.0,1.321586e+06,454,2020-08-26,"2, 4, 6"


In [42]:
#####################################################
# Step 2: Add Times Based on Rates/Changeovers
#####################################################
### requires setup_df, speed_df

# slitter_speed = speed_df.loc[(speed_df['Customer Name'] == customer) &
#          (speed_df['Description'].str.contains(technology)) &
#          (speed_df['Description'].str.contains(color)), 'Slittter Speed (m/min)'].reset_index(drop=True)[0]
slitter_speed = 1300

# add column 'completion date/time'
# for row in schedule, calculate the completion time
sorted_schedule_with_order_info['Completion Date'] = None

### choose starte date/time
# start_date_time = datetime.datetime(2020, 8, 12)
if DEBUG:
    print("start time: {}".format(start_date_time))

### set changeover times
jumbo_change = setup_df.loc[setup_df['Slitter Set-up'] ==
                            'Jumbo roll only']['Time (minutes)'].values[0]
jumbo_change = datetime.timedelta(minutes=jumbo_change)
if DEBUG:
    print("jumbo change: {}".format(jumbo_change))
jumbo_and_knife_change = setup_df.loc[setup_df['Slitter Set-up'] ==
                                      'AMB & Arium']['Time (minutes)'].values[0]
jumbo_and_knife_change = datetime.timedelta(minutes=jumbo_and_knife_change)
if DEBUG:
    print("jumbo and knife change: {}".format(jumbo_and_knife_change))

### print slitter speed and doff length
if DEBUG:
    print("slitter speed (m/min): {}".format(slitter_speed))
    print("doff length (m): {}".format(L))
prior_completion_date_time = start_date_time
match = '\d\d\d'
layout_columns = [i for i in sorted_schedule_with_order_info.columns if re.match(match, i)]

### zero-out the 'prior layout' so algorithm knows jumbo + knife change
prior_layout = sorted_schedule_with_order_info.iloc[0][layout_columns]
for col in prior_layout.index:
    prior_layout[col] = 0
layout_number = 0

schedule_with_change_over = pd.DataFrame()
### transition times are for loading the current jumbo
for row in sorted_schedule_with_order_info.index:

    layout = sorted_schedule_with_order_info.iloc[row][layout_columns]
    if all(prior_layout.fillna(0) == layout.fillna(0)):
        transition_time = jumbo_change
    else:
        transition_time = jumbo_and_knife_change
        layout_number += 1
    if DEBUG:
        print("transition time: {}".format(transition_time))
    run_time = L / slitter_speed
    run_time = datetime.timedelta(minutes=run_time)
    if DEBUG:
        print("run time: {}".format(run_time))
    completion_time = transition_time + run_time
    completion_date_time = prior_completion_date_time + completion_time
    if DEBUG:
        print("completion date/time: {}".format(completion_date_time))

    ### change main df
    sorted_schedule_with_order_info['Completion Date'][row] = completion_date_time
    # sorted_schedule_with_order_info['layout number'][row] = layout_number

    ### with changeover rows
    current_changeover = pd.DataFrame(sorted_schedule_with_order_info.columns)
    current_changeover = current_changeover.set_index(0)
    current_changeover = current_changeover.T
    current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
    current_changeover['Completion Date'] = prior_completion_date_time + transition_time
    current_order = pd.DataFrame(sorted_schedule_with_order_info.iloc[row]).T
    current_combined = current_changeover.append(current_order, sort=False, ignore_index=True)

    schedule_with_change_over = pd.concat([schedule_with_change_over, current_combined])

    prior_completion_date_time = completion_date_time
    prior_layout = layout
    clear_output(wait=True)

schedule_with_change_over = schedule_with_change_over.reset_index(drop=True)

#####################################################
# Step 3: Create Summary With Changeover Rows
#####################################################

master_schedule = pd.DataFrame()
unformatted_schedule = pd.DataFrame()
for index in schedule_with_change_over.index[1::2]: # pass through every jumbo
    order_number = schedule_with_change_over.iloc[index]['Order Number']

    ### these are columns specific to width layouts
    match = '\d\d\d'
    layout_columns = [i for i in schedule_with_change_over.columns if re.match(match, i)]


    deckle_layout =  deckle_order = pd.DataFrame(schedule_with_change_over.iloc[index]).T

    ### these are columns specific to order details
    order_columns = [i for i in deckle_order.columns if (i not in layout_columns)
                     & (i not in ['freq', 'layout number'])]

    ### for order we grab the last row since this is the real 'completion date'
    order = deckle_order[order_columns].reset_index(drop=True).iloc[-1].dropna()

    ### remove widths that are in the deckle opt. but not in the layout
    formula = deckle_layout[layout_columns].reset_index(drop=True).iloc[0].dropna()

    ### number of doffs for the given layout/order
    doffs = doffs_in_jumbo
    read_out = ''
    for i in range(formula.shape[0]-1): #for unique prods
        read_out = read_out + ("{}x{} + ".format(formula.index[i],
                                                formula.iloc[i].astype(int)))
    read_out = read_out + ("{}x{}".format(formula.index[-1],
                                        formula.iloc[-1].astype(int))) #make last string w/o +
    current = pd.DataFrame([read_out, int(doffs), order]).T
    order = pd.DataFrame(order).T
    order = order.reset_index(drop=True)
    current = pd.DataFrame([read_out, int(doffs)]).T
    current.columns = ['Formula', 'Doffs']
    current = current.join(order)

    ### add changeover info
    current_changeover = pd.DataFrame(current.columns)
    current_changeover = current_changeover.set_index(0)
    current_changeover = current_changeover.T
    current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
    current_changeover['Completion Date'] = schedule_with_change_over.iloc[index-1]['Completion Date']
    current_combined = current_changeover.append(current, sort=False, ignore_index=True)
    current_combined

    master_schedule = pd.concat([master_schedule, current_combined], sort=False)
    master_schedule = master_schedule.reset_index(drop=True)

transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 22:23:04.615388


In [43]:
master_schedule

,Formula,Doffs,Total LM Order QTY,Width,Scheduled Ship Date,Order Number,Completion Date
0,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 00:50:00.000000
1,245x1 + 440x1 + 454x2 + 510x1 + 600x1 + 670x2,6,816327,245,2020-08-15,1 and 3,2020-08-01 02:08:27.692308
2,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 02:58:27.692308
3,245x1 + 454x3 + 510x2 + 670x2,6,816327,245,2020-08-15,1 and 3,2020-08-01 04:16:55.384616
4,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 05:06:55.384616
5,440x1 + 454x1 + 510x1 + 600x3 + 670x1,6,816327,245,2020-08-15,1 and 3,2020-08-01 06:25:23.076924
6,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 07:15:23.076924
7,245x5 + 440x2 + 600x2 + 670x1,6,816327,245,2020-08-15,1 and 3,2020-08-01 08:33:50.769232
8,NaN,NaN,NaN,NaN,NaT,Changeover,2020-08-01 09:23:50.769232
9,440x3 + 454x1 + 510x2 + 600x1 + 670x1,6,816327,245,2020-08-15,1 and 3,2020-08-01 10:42:18.461540


In [ ]:
master_knife = optimize_schedule(sol, widths, neckins, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date, 'Time (Knife Changes)')
deckle_time = master_knife.iloc[-1]['Completion Date'] - master_knife.iloc[0]['Completion Date']
print("deckle time, knife changes objective: {}".format(str(deckle_time).split('.')[0]))

In [ ]:
master_late = optimize_schedule(sol, widths, neckins, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date, 'Late Orders')
deckle_time = master_late.iloc[-1]['Completion Date'] - master_late.iloc[0]['Completion Date']
print("deckle time, late orders objective: {}".format(str(deckle_time).split('.')[0]))